In [1]:
# 06/02/25 
# Gabriella Chan
# gabriella.chan@moansh.edu

# This script generates a memory mapped distance matrix using the brainsmash volume function
# It loads an atlas, takes coordinates of valid rois, and calls "volume" to generate the distance matrix

In [1]:
import nibabel as nib
import pandas as pd
import numpy as np
from brainsmash.workbench import volume
from brainsmash.mapgen.eval import sampled_fit
from brainsmash.mapgen.sampled import Sampled


import time
from ttictoc import tic, toc

In [ ]:
# This cell is for Trang

data_dir = './data/bgs_quad/'
out_dir = './results/bgs_quad/'

# Load img
img = nib.load('./data/spmT_0001_quadrant.nii.gz')
img_data = img.get_fdata()
mask = (img_data != 0)

(129821, 3)


In [2]:
data_dir = './data/lh/'
out_dir = './results/lh/'

# Load atlas and brain map
atlas_img = nib.load('/fs03/kg98/gchan/Atlases/Tian/Schaefer_Tian/reordered/Schaefer2018_100Parcels_' +
    '7Networks_order_Tian_Subcortex_S2_MNI152NLin6Asym_1.5mm_reordered.nii.gz')
atlas = atlas_img.get_fdata()
img = nib.load('./data/lme_betas.nii.gz')
img_data = img.get_fdata()

roi_i = 1
roi_j = 66
mask = (atlas >= roi_i) & (atlas <= roi_j)

In [3]:
gen_distmat = False
if gen_distmat:
    tic()
    # Get the 3D coordinates and values of the masked image
    coords = np.column_stack(np.where(mask))
    coord_file = data_dir + 'coordinates.txt'
    np.savetxt(coord_file, coords, fmt='%d')

    gmv = img_data[np.where(mask)]
    brain_map = data_dir + 'lme_betas.txt'
    np.savetxt(brain_map, gmv, fmt='%f')

    # volume is a builtin BrainSMASH function that generates a memory-mapped distance matrix
    # This has a very long runtime
    filenames = volume(coord_file, data_dir)
    toc()
else:
    brain_map = data_dir + 'lme_betas.txt'
    filenames = {"D": data_dir + "distmat.npy", 
                "index": data_dir + "index.npy"
                }

In [5]:
# 13 min l cx knn 1500, 14 min knn 2300

# These are three of the key parameters affecting the variogram fit
kwargs = {'ns': 500,
          'knn': 1500,
          'pv': 60,
          'kernel': 'gaussian',
          'n_jobs': 12
          }

# Running this command will generate a matplotlib figure
# tic()
# sampled_fit(brain_map, filenames['D'], filenames['index'], nsurr=10, **kwargs)
# toc()

In [6]:
tic()
# 20 min 200k voxels
gen = Sampled(x=brain_map, D=filenames['D'], index=filenames['index'], **kwargs)
toc()

1016.867674135603

In [ ]:
# There is no economies of scale in generating multiple surrogate maps
# 4h for 1 null map of 166k voxels

tic()
n_maps = 1
surrogate_maps = gen(n=n_maps)
print(surrogate_maps.shape)
toc()

In [ ]:
def save_map(null_map, out_dir, filename):
    modified_img = nib.Nifti1Image(null_map, img.affine, img.header)
    nib.save(modified_img, out_dir + filename)

In [ ]:
null_map = np.zeros_like(img_data, dtype=float)
if n_maps == 1:
    null_map[mask] = surrogate_maps
    save_map(null_map, out_dir, "test_timing.nii.gz")
else:
    for i in range(n_maps):
        null_map[mask] = surrogate_maps[i]
        save_map(null_map, out_dir, f"null_map_{i}.nii.gz")

# np.savetxt(out_dir + 'mask.csv', mask, delimiter=',', fmt='%d')
# np.savetxt(out_dir + 'surrogate_maps.csv', surrogate_maps, delimiter=',', fmt='%f')